In [1]:
from datasets import load_dataset

dataset = load_dataset("openbookqa", "additional", split=["train", "test"])

In [2]:
dataset[0][0]

{'id': '7-980',
 'question_stem': 'The sun is responsible for',
 'choices': {'text': ['puppies learning new tricks',
   'children growing up and getting old',
   'flowers wilting in a vase',
   'plants sprouting, blooming and wilting'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'D',
 'fact1': 'the sun is the source of energy for physical cycles on Earth',
 'humanScore': 1.0,
 'clarity': 2.0,
 'turkIdAnonymized': 'b356d338b7'}

In [3]:
def get_prompt(dataset, index, perm=[0, 1, 2, 3], has_choice=False):
    json_line = dataset[index]
    question = json_line["question_stem"]
    choices = json_line["choices"]
    choice_texts = choices["text"]
    choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
    candidates = " ".join(
        [
            f"({label if has_choice else ' '}) {text}"
            for text, label in zip(choice_texts, choices["label"])
        ]
    ).replace("\n", " ")
    answer_key = json_line["answerKey"]
    answer_key_idx = ord(answer_key[0]) - ord("A")
    answer_text = choice_texts[answer_key_idx]
    fact = f"{json_line['fact1']}{'' if json_line['fact1'][-1]=='.' else '.'}"
    prompt = f"{fact} {question} \\n {candidates}"
    return prompt, answer_text

In [4]:
from tqdm import tqdm
import pickle
DATASET_NAME='obqa_fact'
for index, dataname in enumerate(['train','test']):
    container=[]
    for ques_index, ques in enumerate(tqdm(dataset[index])):
        container.append(
                get_prompt(dataset[index], ques_index, has_choice=False)
            )
    pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 500/500 [00:00<00:00, 7010.58it/s]
